In the previous tutorials we introduced you to some basic finite fields
$$\mathbb{Z}_p$$
and the collection of polynomials (polynomial rings if we want to use jargon) over them
$$\mathbb{Z}_p[X]$$
While these finite fields are very interesting, they're often not the most convenient space to do cryptographic computations. Part of the reason is that they're kind of exotic. As humans, we don't expect numbers to wrap at 19 or 31 or whatever prime we choose. How about
$$\mathbb{Z}_{10}$$
then? Unfortunately, this isn't a finite field (since 10 factors into 2,5). It turns out that the interesting place to look is "binary" finite fields. What's a binary finite field? Well to give a simplistic explanation, it's any finite field where
$$1 + 1 = 0$$
The simplest example of a binary finite field is 
$$\mathbb{Z}_2$$
Let's play with this finite field a bit.

In [1]:
import starks
from starks.modp import IntegersModP

mod2 = IntegersModP(2)
one = mod2(1)
one + one

0 (mod 2)

Ok, this is neat. Let's play with this a bit more. Can we make an `AND` gate for example? Recall that an `AND` gate implements the logical operator `AND`

![and_gate](http://www.circuitstoday.com/wp-content/uploads/2010/04/2-Input-AND-Gate-Truth-Table.jpg)

Luckily, it turns out that the `AND` gate is really simple to make! It's just a multiplication operation in $\mathbb{Z}_2$.

In [2]:
def AND(x, y):
  return x * y
zero = mod2(0)
one = mod2(1)
print("AND(0, 0) = %s" % str(AND(zero, zero)))
print("AND(0, 1) = %s" % str(AND(zero, one)))
print("AND(1, 0) = %s" % str(AND(one, zero)))
print("AND(1, 1) = %s" % str(AND(one, one)))

AND(0, 0) = 0
AND(0, 1) = 0
AND(1, 0) = 0
AND(1, 1) = 1


This is pretty neat. Note that `AND` is equivalently represented as a simple polynomial in $\mathbb{Z}_2[X, y]$
$$\textrm{and}(x, y) = xy$$
Can we do this for other logical operators? Let's take a look at an `OR` gate
![or_gate](http://hyperphysics.phy-astr.gsu.edu/hbase/Electronic/ietron/or.gif)
This one is a little trickier to write down as a function, but it's not too bad.

In [3]:
def OR(x, y):
  return x + y - x * y
zero = mod2(0)
one = mod2(1)
print("OR(0, 0) = %s" % str(OR(zero, zero)))
print("OR(0, 1) = %s" % str(OR(zero, one)))
print("OR(1, 0) = %s" % str(OR(one, zero)))
print("OR(1, 1) = %s" % str(OR(one, one)))

OR(0, 0) = 0
OR(0, 1) = 1
OR(1, 0) = 1
OR(1, 1) = 1


Take a minute to study this function to understand why it makes sense. Note again that we can write `OR` as a polynomial
$$\textrm{OR}(x, y) = x + y - xy$$
How about `NOT`? Turns out this one is pretty simple to do too.

In [4]:
def NOT(x):
  return (1 - x)
zero = mod2(0)
one = mod2(1)
print("NOT(0) = %s" % str(NOT(zero)))
print("NOT(1) = %s" % str(NOT(one)))

NOT(0) = 1
NOT(1) = 0


This is an interesting result since the three logical operators `OR`, `AND`, and `NOT` are "functionally complete". That means that any boolean circuit can be represented in terms of these operators. Since we've encoded these operators as polynomials over $\mathbb{Z}_2$, we now know we can represent any boolean circuit as a polynomial!

This is neat result, but it's not yet as useful as we'd like it. For one thing, we're operating on $\mathbb{Z}_2$, which is essentially a single bit. What if we want to represent a complex program, which accepts as input 64 bit words? We could represent 64-bit words as lists of 64 elements of $\mathbb{Z}_2$, but this is somewhat awkward. Isn't there a more elegant way to represent such words? How about the set of numbers $\mathbb{Z}/2^{64}$ modulo $2^{64}$? This seems to fit what we're looking for, but we know that it's not a finite field. Division doesn't work since $2^{64}$ has many prime factors.

It seems like we're stuck here. Luckily, there is a way to construct a finite field of size $2^{64}$. The technical construction is a little complex, so we're going to start by just putting some simple code examples in front of you to start.

In [5]:
from starks.polynomial import polynomials_over
from starks.finitefield import FiniteField

p = 2
m = 64
Zp = IntegersModP(p)
polysOver = polynomials_over(Zp)

#x^64 + x^4 + x^3 + x + 1 is irreducible (in fact primitive). Don't worry about what this means yet 
coefficients = [Zp(0)] * 65
coefficients[0] = Zp(1)
coefficients[1] = Zp(1)
coefficients[3] = Zp(1)
coefficients[4] = Zp(1)
coefficients[64] = Zp(1)
poly = polysOver(coefficients)
field = FiniteField(p, m, polynomialModulus=poly)

This probably seems a little mysterious. What is this business with a primitive polynomial? What do polynomials have to do with anything here? You'll learn more about the details of these polynomials in the next part of this tutorial, but for now, let's treat these objects as givens and just play with some examples. 

The first thing to note is that elements of a finite field are polynomials. Let's start with some really basic definitions.

In [7]:
A = field(polysOver([1,1,1]))
A

1 + 1 x^1 + 1 x^2 ∈ F_{2^64}

In [8]:
B = field(polysOver([1,0,1,1]))
B

1 + 0 x^1 + 1 x^2 + 1 x^3 ∈ F_{2^64}

For now, don't worry too much about the precise definitions of `A` and `B`. We'll come back and explain what these mean in the next tutorial. But let's just see that we can do all the usual things we'd expect to be able to do in a finite field with them.

In [9]:
A + B

0 + 1 x^1 + 0 x^2 + 1 x^3 ∈ F_{2^64}

In [10]:
A - B

0 + 1 x^1 + 0 x^2 + 1 x^3 ∈ F_{2^64}

In [11]:
A * B

1 + 1 x^1 + 0 x^2 + 0 x^3 + 0 x^4 + 1 x^5 ∈ F_{2^64}

In [12]:
A / B

1 + 0 x^1 + 0 x^2 + 0 x^3 + 1 x^4 + 0 x^5 + 0 x^6 + 1 x^7 + 0 x^8 + 1 x^9 + 1 x^10 + 1 x^11 + 0 x^12 + 0 x^13 + 1 x^14 + 0 x^15 + 1 x^16 + 1 x^17 + 1 x^18 + 0 x^19 + 0 x^20 + 1 x^21 + 0 x^22 + 1 x^23 + 1 x^24 + 1 x^25 + 0 x^26 + 0 x^27 + 1 x^28 + 0 x^29 + 1 x^30 + 1 x^31 + 1 x^32 + 0 x^33 + 0 x^34 + 1 x^35 + 0 x^36 + 1 x^37 + 1 x^38 + 1 x^39 + 0 x^40 + 0 x^41 + 1 x^42 + 0 x^43 + 1 x^44 + 1 x^45 + 1 x^46 + 0 x^47 + 0 x^48 + 1 x^49 + 0 x^50 + 1 x^51 + 1 x^52 + 1 x^53 + 0 x^54 + 0 x^55 + 1 x^56 + 0 x^57 + 1 x^58 + 1 x^59 + 1 x^60 + 0 x^61 + 0 x^62 + 1 x^63 ∈ F_{2^64}

We can add, subtract, multiply and divide these quantities. The precise mechanics of what these operations will be explained in more detail soon, but for now you can try running a few different operations. 

How should you be thinking about binary finite fields? They seem to be these strange objects that are complicated polynomials. How does this tie into our earlier discussion about binary operations in $\mathbb{Z}_2$? Well, we now have a way to turn 64 bit words into these binary finite field objects. There's a lot of questions this linkage brings up. How do the addition, subtractions, multiplication, and division operations shown here compare with the "usual" notions that that processors do? It turns out this question is a little subtle. There are two different sets of arithmetic operations here, one belonging to the finite field, another which is the "usual" arithmetic. This is shading into research, so we won't say too much more on this topic yet.